<a href="https://colab.research.google.com/github/omkar-salunke/accounts/blob/main/both_tally_daybook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
# prompt: list of files in the path in python

import os
import pandas as pd
import xml.etree.ElementTree as ET
import pandas as pd
from openpyxl import load_workbook

def extract_element_data(element):
    element_data = {}
    # If the element has no children, return its text
    if len(element) == 0:
        return element.text

    # Otherwise, go deeper into the element's children
    for child in element:
        # Recursively extract data for each child element
        child_data = extract_element_data(child)
        element_data[child.tag] = child_data
    return element_data

# Function to flatten nested dictionaries (e.g., 'amount': {'P': {'T': 100}})
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Function to recursively search for all 'voucher' elements and extract their data
def find_all_vouchers(element, voucher_tag="VOUCHER"):
    vouchers = []

    # If the current element is a 'voucher', collect its data
    if element.tag == voucher_tag:
        voucher_dict = {}
        # Collect data for all child elements within the voucher
        for sub_child in element:
            extracted_data = extract_element_data(sub_child)
            voucher_dict[sub_child.tag] = extracted_data

        # Flatten the voucher dictionary to handle nested elements
        voucher_dict_flattened = flatten_dict(voucher_dict)
        vouchers.append(voucher_dict_flattened)

    # Recursively search in all children of the current element
    for child in element:
        vouchers.extend(find_all_vouchers(child, voucher_tag))

    return vouchers

# Function to convert the XML to a DataFrame by extracting all vouchers
def xml_to_dataframe_vouchers(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Find all 'voucher' elements in the XML
    voucher_list = find_all_vouchers(root)

    # Create a DataFrame from the list of voucher dictionaries
    df = pd.DataFrame(voucher_list)
    return df




In [166]:
# df_bank_match_soe = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='SOE 2024-25')
# df_bank_match_overview_soe = df_bank_match_soe.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

# df_bank_match_me = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='ME 2024-25')
# df_bank_match_overview_me = df_bank_match_me.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

In [167]:
soe_list = '/content/drive/MyDrive/tally_export/SOE/'
me_list = '/content/drive/MyDrive/tally_export/ME/'
files_soe = os.listdir('/content/drive/MyDrive/tally_export/SOE/')
files_me = os.listdir('/content/drive/MyDrive/tally_export/ME/')

In [168]:

# xml_to_dataframe_vouchers('/content/drive/MyDrive/tally_export/ME/DayBook.xml')

In [169]:
req_cols = ['VOUCHERNUMBER','DATE','GUID','VOUCHERTYPENAME','PARTYLEDGERNAME','PARTYGSTIN','CLASSNAME','PARTYNAME','CONSIGNEEGSTIN','PERSISTEDVIEW','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME','ALLINVENTORYENTRIES.LIST.RATE','ALLINVENTORYENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.LEDGERNAME','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.AMOUNT','LEDGERENTRIES.LIST.LEDGERNAME','LEDGERENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','INVOICEORDERLIST.LIST.BASICORDERDATE','INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO','REFERENCEDATE','REFERENCE','CONSIGNEEPINNUMBER','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD','ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS','ALLLEDGERENTRIES.LIST.LEDGERNAME','ALLLEDGERENTRIES.LIST.AMOUNT','NARRATION']
df_raw_collect_soe = pd.DataFrame()
for xt in files_soe:
  path_ct = soe_list
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)
  df_raw_f_ref = pd.DataFrame(df_raw_f)[req_cols]
  print(df_raw_f_ref.loc[0,'DATE'],df_raw_f_ref.loc[len(df_raw_f_ref)-1,'DATE'])
  df_raw_collect_soe = pd.concat([df_raw_collect_soe, df_raw_f_ref]).reset_index(drop=True)
df_raw_collect_soe

df_raw_soe = df_raw_collect_soe.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_soe['date_column'] = pd.to_datetime(df_raw_soe['DATE'], format='%Y%m%d')
df_raw_soe = df_raw_soe.sort_values('date_column')
df_raw_soe['date_column_month'] = df_raw_soe['date_column'].dt.month_name()
df_raw_soe['act_qty'] = df_raw_soe['ALLINVENTORYENTRIES.LIST.ACTUALQTY'].str.extract(r'([\d.]+)').astype(float)
df_raw_soe_sales = df_raw_soe[df_raw_soe['VOUCHERTYPENAME'].isin(['Sales Gst', 'SALES GST'])]
df_raw_soe_sales_sch = df_raw_soe_sales.groupby(['date_column_month','PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'])['act_qty'].sum()

df_schedule_merge_soe = pd.DataFrame(df_raw_soe_sales_sch).pivot_table(index=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],columns='date_column_month',values='act_qty',aggfunc='sum').reset_index()
df_schedule_merge_soe['FROM']="SOE"

# df_schedule = pd.read_excel("/content/drive/MyDrive/tally_export/schedule.xlsx",sheet_name = 'SOE')
# df_schedule_merge_soe = pd.merge(df_schedule,df_raw_soe_sales_schedule,on=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],how='right')
# df_schedule_merge.to_excel(,sheet_name = 'SOE',index=False)
df_schedule_merge_soe

# df_raw_soe_sales.sort_values('date_column')



20250502 20250519
20250502 20250519
20250502 20250523
20250401 20250526
20250502 20250530


date_column_month,PARTYLEDGERNAME,ALLINVENTORYENTRIES.LIST.STOCKITEMNAME,April,May,FROM
0,AKAR INDUSTRY,2MM PTO CAP,15000.0,17715.0,SOE
1,AKAR INDUSTRY,ANCHOR PLATE DIFF.LOCK LEVER,3350.0,450.0,SOE
2,AKAR INDUSTRY,BKT BUMPER,4150.0,5890.0,SOE
3,AKAR INDUSTRY,BLANK OF LOCK PLATE DIFF.RING GEAR,17005.0,NaN,SOE
4,AKAR INDUSTRY,DIFF.CASE LOCK PLATE,25052.0,33020.0,SOE
5,AKAR INDUSTRY,DUST CAP-F1138613,10970.0,8239.0,SOE
6,AKAR INDUSTRY,HOOD HANDLE CLAMP,NaN,4036.0,SOE
7,AKAR INDUSTRY,LOCK PLATE BULL GEAR,5940.0,25500.0,SOE
8,AKAR INDUSTRY,ROCKSHAFT WASHER,2220.0,20420.0,SOE
9,AKAR INDUSTRY,SS WASHER -4MM,NaN,10755.0,SOE


In [170]:
zoho_col_list = ['Invoice Number',
'Estimate Number',
'Invoice Date',
'Invoice Status',
'Customer Name',
'GST Treatment',
'TCS Tax Name',
'TCS Percentage',
'TCS Amount',
'Nature Of Collection',
'TCS Payable Account',
'TCS Receivable Account',
'GST Identification Number (GSTIN)',
'TDS Name',
'TDS Percentage',
'TDS Section Code',
'TDS Amount',
'Place of Supply',
'PurchaseOrder',
'Expense Reference ID',
'Payment Terms',
'Payment Terms Label',
'Due Date',
'Expected Payment Date',
'Sales person',
'Shipping Charge Tax Name',
'Shipping Charge Tax Type',
'Shipping Charge Tax %',
'Shipping Charge',
'Shipping Charge Tax Exemption Code',
'Shipping Charge SAC Code',
'Currency Code',
'Exchange Rate',
'Account',
'Item Name',
'SKU',
'Item Desc',
'Item Type',
'HSN/SAC',
'Quantity',
'Usage unit',
'Item Price',
'Item Tax Exemption Reason',
'Is Inclusive Tax',
'Item Tax',
'Item Tax Type',
'Item Tax %',
'Reverse Charge Tax Name',
'Reverse Charge Tax Rate',
'Reverse Charge Tax Type',
'Project Name',
'Supply Type',
'Discount Type',
'Is Discount Before Tax',
'Entity Discount Percent',
'Entity Discount Amount',
'Discount',
'Discount Amount',
'Adjustment',
'Adjustment Description',
'E-Commerce Operator Name',
'E-Commerce Operator GSTIN',
'PayPal',
'Razorpay',
'Partial Payments',
'Template Name',
'Notes',
'Terms & Conditions',
'Branch Name']

In [171]:
# req_cols = ['DATE','GUID','VOUCHERTYPENAME','PARTYLEDGERNAME','PARTYGSTIN','CLASSNAME','PARTYNAME','CONSIGNEEGSTIN','PERSISTEDVIEW','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME','ALLINVENTORYENTRIES.LIST.RATE','ALLINVENTORYENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.LEDGERNAME','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.AMOUNT','LEDGERENTRIES.LIST.LEDGERNAME','LEDGERENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','INVOICEORDERLIST.LIST.BASICORDERDATE','INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO','REFERENCEDATE','REFERENCE','CONSIGNEEPINNUMBER','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD','ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS','ALLLEDGERENTRIES.LIST.LEDGERNAME','ALLLEDGERENTRIES.LIST.AMOUNT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.DATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.INSTRUMENTDATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.NAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT','NARRATION']
df_raw_collect_soe = pd.DataFrame()
for xt in files_me:
  path_ct = me_list
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)
  df_raw_f_ref = pd.DataFrame(df_raw_f)[req_cols]
  df_raw_collect_soe = pd.concat([df_raw_collect_soe, df_raw_f_ref]).reset_index(drop=True)
# df_raw_collect_soe
df_raw_soe = df_raw_collect_soe.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_soe['date_column'] = pd.to_datetime(df_raw_soe['DATE'], format='%Y%m%d')
df_raw_soe = df_raw_soe.sort_values('date_column')
df_raw_soe['date_column_month'] = df_raw_soe['date_column'].dt.month_name()
df_raw_soe['act_qty'] = df_raw_soe['ALLINVENTORYENTRIES.LIST.ACTUALQTY'].str.extract(r'([\d.]+)').astype(float)

df_raw_me_sales = df_raw_soe[df_raw_soe['VOUCHERTYPENAME'].isin(['Sales Gst', 'SALES GST'])]
df_raw_me_sales_sch = df_raw_me_sales.groupby(['date_column_month','PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'])['act_qty'].sum()

df_schedule_merge_me = pd.DataFrame(df_raw_me_sales_sch).pivot_table(index=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],columns='date_column_month',values='act_qty',aggfunc='sum').reset_index()
df_schedule_merge_me['FROM']="ME"

# df_schedule = pd.read_excel("/content/drive/MyDrive/tally_export/schedule.xlsx",sheet_name = 'ME')
# df_raw_soe_sales_schedule
# df_schedule_merge_me = pd.merge(df_schedule,df_raw_soe_sales_schedule,on=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],how='right')

# writer = pd.ExcelWriter("/content/drive/MyDrive/tally_export/schedule_overview.xlsx", engine = 'openpyxl')
# df_schedule_merge_soe.to_excel(writer, sheet_name = 'SOE')
# df_schedule_merge_me.to_excel(writer, sheet_name = 'ME')
# writer.close()
df_raw_me_sales.sort_values('date_column')


,VOUCHERNUMBER,DATE,GUID,VOUCHERTYPENAME,PARTYLEDGERNAME,PARTYGSTIN,CLASSNAME,PARTYNAME,CONSIGNEEGSTIN,PERSISTEDVIEW,...,CONSIGNEEPINNUMBER,ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE,ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD,ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS,ALLLEDGERENTRIES.LIST.LEDGERNAME,ALLLEDGERENTRIES.LIST.AMOUNT,NARRATION,date_column,date_column_month,act_qty
0,25-26/001,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-00001995,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,Sales Taxable,Cess,NaN,NaN,NaN,NaN,2025-04-01,April,728.0
1,25-26/002,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-00001996,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,242.0
6,25-26/007,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-0000199c,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,300.0
2,25-26/003,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-00001997,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,200.0
3,25-26/004,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-00001999,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,Cess,NaN,NaN,NaN,NaN,2025-04-01,April,303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,25-26/358,20250529,9560e508-101c-4d25-b11f-69c1f97a4f64-00001ba7,SALES GST,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-29,May,10000.0
444,25-26/359,20250529,9560e508-101c-4d25-b11f-69c1f97a4f64-00001ba8,SALES GST,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-29,May,6720.0
441,25-26/356,20250529,9560e508-101c-4d25-b11f-69c1f97a4f64-00001ba5,SALES GST,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,...,NaN,NaN,Cess,NaN,NaN,NaN,NaN,2025-05-29,May,4000.0
446,25-26/361,20250529,9560e508-101c-4d25-b11f-69c1f97a4f64-00001ba9,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,27AADCV3787B1Z8,SALES GST 18%,V S AUTO TECH PRIVATE LIMITED,27AADCV3787B1Z8,Invoice Voucher View,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-29,May,5150.0


In [172]:
df_schedule = pd.read_excel("/content/drive/MyDrive/tally_export/schedule_overview.xlsx",sheet_name = 'Planned_Schedule')
# df_raw_soe_sales_schedule
df_schedule

,FROM,TO,part_tally_name,Schedule_May
0,SOE,AKAR INDUSTRY,2MM PTO CAP,16000.0
1,SOE,AKAR INDUSTRY,ANCHOR PLATE DIFF.LOCK LEVER,NaN
2,SOE,AKAR INDUSTRY,BKT BUMPER,NaN
3,SOE,AKAR INDUSTRY,BLANK OF LOCK PLATE DIFF.RING GEAR,NaN
4,SOE,AKAR INDUSTRY,DIFF.CASE LOCK PLATE,NaN
...,...,...,...,...
97,ME,SHREE SAI ENGINEERING,SUPPORT BKT RH FOR SIDE SEALING P.NO.007653373V01,5000.0
98,ME,SHREE SAI ENGINEERING,TOOL BOX BKT LH PART NO.007544527Y92,1000.0
99,ME,SHREE SAI ENGINEERING,TOOL BOX BKT RH PART NO.007532393Y92,1000.0
100,ME,V S AUTO TECH PRIVATE LIMITED,QUARDANT SUPPORT BKT.007208467Y1,NaN


In [173]:
df_concat = pd.concat([df_schedule_merge_me,df_schedule_merge_soe]).rename(columns={"PARTYLEDGERNAME":"TO","ALLINVENTORYENTRIES.LIST.STOCKITEMNAME":"part_tally_name"})
from datetime import date
df_schedule_merge_relate = pd.merge(df_schedule,df_concat,on=['FROM','TO','part_tally_name'],how='right')
df_schedule_merge_relate
month = "May"
df_schedule_merge_relate['balance_'+month+'_'+str(date.today())] = df_schedule_merge_relate['Schedule_'+month]-df_schedule_merge_relate[month]
df_schedule_merge_relate = df_schedule_merge_relate[['FROM','TO','part_tally_name']+[month]+['Schedule_'+month]+['balance_'+month+'_'+str(date.today())]]
df_schedule_merge_relate

,FROM,TO,part_tally_name,May,Schedule_May,balance_May_2025-05-30
0,ME,AKAR INDUSTRY,GUSSET GUIDE MOUNTING,15295.0,15000.0,-295.0
1,ME,AKAR INDUSTRY,HARNESS MTG BKT CLIP,30500.0,NaN,NaN
2,ME,AKAR INDUSTRY,LUG 1 CHANNEL PART NO.005556488R1,3120.0,NaN,NaN
3,ME,AKAR INDUSTRY,LUG 2 CHANNEL PART.NO.005556487R1,3770.0,NaN,NaN
4,ME,AKAR INDUSTRY,LUG 3 CHANNEL PART NO. 005556156R1,5690.0,NaN,NaN
...,...,...,...,...,...,...
113,SOE,V S AUTO TECH PRIVATE LIMITED,PC LEVER CPTE M6 PART.NO.P.NO.007217353B91,4475.0,NaN,NaN
114,SOE,V S AUTO TECH PRIVATE LIMITED,PC LEVER CRANK CPTE 10MM,1494.0,NaN,NaN
115,SOE,V.S.AUTO TECH PRIVATE LIMITED,BATTERY COVER BKT,3500.0,NaN,NaN
116,SOE,V.S.AUTO TECH PRIVATE LIMITED,FLANGE QUADRANT SIDE,2615.0,NaN,NaN


In [174]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

file_path = "/content/drive/MyDrive/tally_export/schedule_overview.xlsx"

# Load existing workbook
wb = load_workbook(filename=file_path)

# Define a function to clear and update a sheet
def update_sheet(sheet_name, df):
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        # Clear existing cells (except formats)
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, max_col=ws.max_column):
            for cell in row:
                cell.value = None
    else:
        ws = wb.create_sheet(title=sheet_name)

    # Write new data
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

# Update only the target sheets
update_sheet('SOE', df_schedule_merge_soe)
update_sheet('ME', df_schedule_merge_me)
update_sheet(month, df_schedule_merge_relate)

# Save to a temp file then replace the original (for GDrive safety)
temp_path = "/content/temp_schedule_overview.xlsx"
wb.save(temp_path)

import shutil
shutil.copy(temp_path, file_path)


'/content/drive/MyDrive/tally_export/schedule_overview.xlsx'

In [175]:
df_raw_soe_sales['date_column_zoho'] = pd.to_datetime(df_raw_soe_sales['DATE'], format='%Y%m%d')
df_raw_soe_sales['date_column_zoho'] = df_raw_soe_sales['date_column_zoho'].dt.strftime('%Y-%m-%d')
df_raw_soe_sales['GST Treatment']="business_gst"
df_raw_soe_sales["Place of Supply"] = "MH"
df_raw_soe_sales["Payment Terms"] = 45
df_raw_soe_sales["Account"] = "Sales"
df_raw_soe_sales["Item Type"] = "goods"
df_raw_soe_sales["HSN/SAC"] = "87081010"
df_raw_soe_sales["Supply Type"] = "Taxable"

df_raw_soe_sales['Invoice Status'] = "open"
df_raw_soe_sales['Item Tax Type'] = "Tax Group"
# try:
df_raw_soe_sales['tax_perc1'] = df_raw_soe_sales['CLASSNAME'].str.extract(r'(\d+)')
df_raw_soe_sales['tax_perc2'] = df_raw_soe_sales['LEDGERENTRIES.LIST.LEDGERNAME'].str.extract(r'(\d+)').fillna(0).astype(int)*2

df_raw_soe_sales['tax_perc'] = df_raw_soe_sales['tax_perc1'].fillna(df_raw_soe_sales['tax_perc2']).astype(int)

#
df_raw_soe_sales['Item Tax'] = "GST"+df_raw_soe_sales['tax_perc'].astype(str)
df_raw_soe_sales['Item Price']=df_raw_soe_sales['ALLINVENTORYENTRIES.LIST.RATE'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

df_raw_soe_sales_rename = df_raw_soe_sales.rename(columns = {"VOUCHERNUMBER":"Invoice Number","date_column_zoho":"Invoice Date","PARTYGSTIN":"GST Identification Number (GSTIN)","INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO":"PurchaseOrder",
                                   "tax_perc":"Item Tax %","ALLINVENTORYENTRIES.LIST.STOCKITEMNAME":"Item Name","act_qty":"Quantity",
                                   "PARTYLEDGERNAME":"Customer Name"})

req_columns = []
for xl in list(df_raw_soe_sales_rename.columns):
  if xl in zoho_col_list:
    req_columns.append(xl)

df_raw_soe_sales_rename[req_columns].to_csv("/content/drive/MyDrive/tally_export/SOE_zoho_sales_invoice.csv",index=False)

In [176]:
df_raw_me_sales['date_column_zoho'] = pd.to_datetime(df_raw_me_sales['DATE'], format='%Y%m%d')
df_raw_me_sales['date_column_zoho'] = df_raw_me_sales['date_column_zoho'].dt.strftime('%Y-%m-%d')
df_raw_me_sales['GST Treatment']="business_gst"
df_raw_me_sales["Place of Supply"] = "MH"
df_raw_me_sales["Payment Terms"] = 45
df_raw_me_sales["Account"] = "Sales"
df_raw_me_sales["Item Type"] = "goods"
df_raw_me_sales["HSN/SAC"] = "87081010"
df_raw_me_sales["Supply Type"] = "Taxable"

df_raw_me_sales['Invoice Status'] = "open"
df_raw_me_sales['Item Tax Type'] = "Tax Group"
# try:
df_raw_me_sales['tax_perc1'] = df_raw_me_sales['CLASSNAME'].str.extract(r'(\d+)')
df_raw_me_sales['tax_perc2'] = df_raw_me_sales['LEDGERENTRIES.LIST.LEDGERNAME'].str.extract(r'(\d+)').fillna(0).astype(int)*2

df_raw_me_sales['tax_perc'] = df_raw_me_sales['tax_perc1'].fillna(df_raw_me_sales['tax_perc2']).astype(int)

#
df_raw_me_sales['Item Tax'] = "GST"+df_raw_me_sales['tax_perc'].astype(str)
df_raw_me_sales['Item Price']=df_raw_me_sales['ALLINVENTORYENTRIES.LIST.RATE'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

df_raw_me_sales_rename = df_raw_me_sales.rename(columns = {"VOUCHERNUMBER":"Invoice Number","date_column_zoho":"Invoice Date","PARTYGSTIN":"GST Identification Number (GSTIN)","INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO":"PurchaseOrder",
                                   "tax_perc":"Item Tax %","ALLINVENTORYENTRIES.LIST.STOCKITEMNAME":"Item Name","act_qty":"Quantity",
                                   "PARTYLEDGERNAME":"Customer Name"})

req_columns = []
for xl in list(df_raw_me_sales_rename.columns):
  if xl in zoho_col_list:
    req_columns.append(xl)

df_raw_me_sales_rename[req_columns].to_csv("/content/drive/MyDrive/tally_export/ME_zoho_sales_invoice.csv",index=False)

<ipython-input-176-daac481d7437>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_me_sales['date_column_zoho'] = pd.to_datetime(df_raw_me_sales['DATE'], format='%Y%m%d')
<ipython-input-176-daac481d7437>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_me_sales['date_column_zoho'] = df_raw_me_sales['date_column_zoho'].dt.strftime('%Y-%m-%d')
<ipython-input-176-daac481d7437>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [177]:
# def add_req_fields(df):
#   df['party_name'] = df[['PARTYNAME', 'ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING', 'ALLLEDGERENTRIES.LIST.LEDGERNAME']].bfill(axis=1).iloc[:, 0]
#   df['tax_out_1'] = df[['LEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
#   df['tax_out_2'] = df[['LEDGERENTRIES.LIST.VATEXPAMOUNT']].bfill(axis=1).iloc[:, 0]

#   # df[''] = df[''].astype(float)
#   df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'] = df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'].astype(float)
#   df['tax_out_1'] = df['tax_out_1'].astype(float)
#   df['tax_out_2'] = df['tax_out_2'].astype(float)

#   df['amount_coalesce'] = df[['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT', 'ALLLEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
#   df['amount_coalesce'] = df['amount_coalesce'].astype(float)
#   df['tax_coalesce'] = df[['tax_out_1', 'tax_out_2']].sum(axis=1)
#   df['total_coalesce'] = df[['tax_coalesce', 'amount_coalesce']].sum(axis=1)

#   df = df[['party_name','total_coalesce','amount_coalesce','tax_coalesce','tax_out_1','tax_out_2']+req_cols]
#   return df

In [178]:
# df_me = add_req_fields(df_raw_me)
# df_me['DATE'] = pd.to_datetime(df_me['DATE'])
# df_me['month_of_day'] =df_me['DATE'].dt.month
# df_me['quarter_of_day'] =df_me['DATE'].dt.quarter

# df_soe = add_req_fields(df_raw_soe)
# df_soe['DATE'] = pd.to_datetime(df_soe['DATE'])
# df_soe['month_of_day'] =df_soe['DATE'].dt.month
# df_soe['quarter_of_day'] =df_soe['DATE'].dt.quarter


In [179]:

# df_soe['total_coalesce'] = df_soe['total_coalesce'].astype('float')
# df_summary_all_soe = pd.pivot_table(df_soe, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
# df_summary_all_soe['balance'] = 0
# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='TIRUMALA MULTI TECHNOLOGIES'].index,'balance'] = 467646
# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='SAVITA AUTO INDUSTRIES (N)'].index,'balance'] = 195853
# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='PATIL-N-PATIL ENGINEERING	'].index,'balance'] = 55660
# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='NANDAMURI TECHNOS'].index,'balance'] = 35255
# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='V.S.AUTO TECH PRIVATE LIMITED'].index,'balance'] = 4699956
# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='A.B.ENGINEERING WORKS'].index,'balance'] = 11598

# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='AKAR INDUSTRY'].index,'balance'] = 317899

# df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='SURYA AUTO INDUSTRIES'].index,'balance'] = 23165


# df_summary_all_soe = df_summary_all_soe.fillna(0)
# df_summary_all_soe['total_due']=df_summary_all_soe[['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note',
#        'Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance']].sum(axis=1)

# df_me['total_coalesce'] = df_me['total_coalesce'].astype('float')
# df_summary_all_me = pd.pivot_table(df_me, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
# df_summary_all_me['balance'] = 0
# df_summary_all_me = df_summary_all_me.fillna(0)
# df_summary_all_me['total_due']=df_summary_all_me[[ 'Contra','Delivery Note','Debit Note','Sales Gst', 'Journal', 'Payment', 'Purchase', 'Receipt','Sales Gst','balance']].sum(axis=1)

In [180]:
# terms = ['SHREE SAI ENGINEERING', 'XYZ COMPANY'] # Function to match and replace terms def match_and_replace(transaction): for term in terms: if term.replace(" ", "").lower() in transaction.replace(" ", "").lower(): return term return transaction # Or return a default value if no match is found # Apply function to the column df['renamed_column'] = df['transaction_column'

In [181]:
# # df_bank_match
# df_summary_all_bank_soe = pd.merge(df_summary_all_soe,df_bank_match_overview_soe,on=['party_name'],how='outer')
# df_summary_all_bank_soe = df_summary_all_bank_soe.sort_values(by='SALES GST', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='SALES GST', ascending=False).reset_index(drop=True)

# # df_bank_match
# df_summary_all_bank_me = pd.merge(df_summary_all_me,df_bank_match_overview_me,on=['party_name'],how='outer')
# df_summary_all_bank_me = df_summary_all_bank_me.sort_values(by='Sales Gst', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='Sales Gst', ascending=False).reset_index(drop=True)



In [182]:

# df_sales = df_soe[df_soe['VOUCHERTYPENAME']=='SALES GST'].reset_index(drop=True)
# # df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
# df_sales_g = df_sales.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"total_sales","PARTYLEDGERNAME":"party_name"})
# date_45d = max(df_sales['DATE']) - pd.Timedelta(days=45)
# due45 = df_sales[df_sales['DATE'] > date_45d]
# due45_g = due45.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_45d","PARTYLEDGERNAME":"party_name"})

# date_60d = max(df_sales['DATE']) - pd.Timedelta(days=60)
# due60 = df_sales[df_sales['DATE'] > date_60d]
# due60_g = due60.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_60d","PARTYLEDGERNAME":"party_name"})

# date_90d = max(df_sales['DATE']) - pd.Timedelta(days=90)
# due90 = df_sales[df_sales['DATE'] > date_90d]
# due90_g = due90.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_90d","PARTYLEDGERNAME":"party_name"})

# # df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe,df_sales_g,on=['party_name'],how='left')
# df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe,due45_g,on=['party_name'],how='left')
# df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe_due,due60_g,on=['party_name'],how='left')
# df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe_due,due90_g,on=['party_name'],how='left')
# df_summary_all_bank_soe_due = df_summary_all_bank_soe_due.fillna(0)
# df_summary_all_bank_soe_due['true_credit'] = df_summary_all_bank_soe_due['Credit']-df_summary_all_bank_soe_due['balance']
# # df_summary_all_bank_soe_due

In [183]:

# df_sales = df_me[df_me['VOUCHERTYPENAME']=='Sales Gst'].reset_index(drop=True)
# # df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
# df_sales_g = df_sales.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"total_sales","PARTYLEDGERNAME":"party_name"})
# date_45d = max(df_sales['DATE']) - pd.Timedelta(days=45)
# due45 = df_sales[df_sales['DATE'] > date_45d]
# due45_g = due45.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_45d","PARTYLEDGERNAME":"party_name"})

# date_60d = max(df_sales['DATE']) - pd.Timedelta(days=60)
# due60 = df_sales[df_sales['DATE'] > date_60d]
# due60_g = due60.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_60d","PARTYLEDGERNAME":"party_name"})

# date_90d = max(df_sales['DATE']) - pd.Timedelta(days=90)
# due90 = df_sales[df_sales['DATE'] > date_90d]
# due90_g = due90.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_90d","PARTYLEDGERNAME":"party_name"})

# # df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me,df_sales_g,on=['party_name'],how='left')
# df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me,due45_g,on=['party_name'],how='left')
# df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me_due,due60_g,on=['party_name'],how='left')
# df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me_due,due90_g,on=['party_name'],how='left')
# df_summary_all_bank_me_due = df_summary_all_bank_me_due.fillna(0)
# df_summary_all_bank_me_due['true_credit'] = df_summary_all_bank_me_due['Credit']-df_summary_all_bank_me_due['balance']
# # df_summary_all_bank_me_due

In [184]:
# import numpy as np
# columns_to_round_soe = ['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note','Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance','total_due', 'Debit', 'Credit','Sale_45d','Sale_60d','Sale_90d']
# df_summary_all_bank_soe_due[columns_to_round_soe] = df_summary_all_bank_soe_due[columns_to_round_soe].apply(np.ceil)

# columns_to_round_me = ['Contra', 'Debit Note', 'Delivery Note', 'Journal','Payment', 'Purchase', 'Receipt', 'Sales Gst', 'balance', 'total_due','Debit', 'Credit','Sale_45d','Sale_60d','Sale_90d']
# df_summary_all_bank_me_due[columns_to_round_me] = df_summary_all_bank_me_due[columns_to_round_me].apply(np.ceil)

In [185]:
# import pandas as pd
# from openpyxl import load_workbook
# from datetime import datetime

# # Define file path and sheet name
# file_path = "/content/drive/My Drive/2024_25/daybook.xlsx"
# original_sheet_name = "SOE_overview"

# # Get the current date to use in the renamed sheet name
# date_str = datetime.now().strftime("%Y_%m_%d")
# renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# # Load the workbook and rename the sheet if it exists
# workbook = load_workbook(file_path)

# if original_sheet_name in workbook.sheetnames:
#     # Rename the existing sheet
#     sheet = workbook[original_sheet_name]
#     sheet.title = renamed_sheet_name
#     sheet.sheet_state = 'hidden'  # Hide the sheet

# # Save the workbook after renaming and hiding the sheet
# workbook.save(file_path)

# # Write the new data to the original sheet name
# with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
#     df_summary_all_bank_soe_due.to_excel(writer, sheet_name=original_sheet_name, index=False)


In [186]:
# original_sheet_name = "SOE_raw_xml"

# # Get the current date to use in the renamed sheet name
# date_str = datetime.now().strftime("%Y_%m_%d")
# renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# # Load the workbook and rename the sheet if it exists
# workbook = load_workbook(file_path)

# if original_sheet_name in workbook.sheetnames:
#     # Rename the existing sheet
#     sheet = workbook[original_sheet_name]
#     sheet.title = renamed_sheet_name
#     sheet.sheet_state = 'hidden'  # Hide the sheet

# # Save the workbook after renaming and hiding the sheet
# workbook.save(file_path)

# # Write the new data to the original sheet name
# with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
#     df_soe.to_excel(writer, sheet_name=original_sheet_name, index=False)

In [187]:
# original_sheet_name = "ME_overview"

# # Get the current date to use in the renamed sheet name
# date_str = datetime.now().strftime("%Y_%m_%d")
# renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# # Load the workbook and rename the sheet if it exists
# workbook = load_workbook(file_path)

# if original_sheet_name in workbook.sheetnames:
#     # Rename the existing sheet
#     sheet = workbook[original_sheet_name]
#     sheet.title = renamed_sheet_name
#     sheet.sheet_state = 'hidden'  # Hide the sheet

# # Save the workbook after renaming and hiding the sheet
# workbook.save(file_path)

# # Write the new data to the original sheet name
# with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
#     df_summary_all_bank_me_due.to_excel(writer, sheet_name=original_sheet_name, index=False)

In [188]:
# original_sheet_name = "ME_raw_xml"

# # Get the current date to use in the renamed sheet name
# date_str = datetime.now().strftime("%Y_%m_%d")
# renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# # Load the workbook and rename the sheet if it exists
# workbook = load_workbook(file_path)

# if original_sheet_name in workbook.sheetnames:
#     # Rename the existing sheet
#     sheet = workbook[original_sheet_name]
#     sheet.title = renamed_sheet_name
#     sheet.sheet_state = 'hidden'  # Hide the sheet

# # Save the workbook after renaming and hiding the sheet
# workbook.save(file_path)

# # Write the new data to the original sheet name
# with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
#     df_me.to_excel(writer, sheet_name=original_sheet_name, index=False)

In [189]:
# original_sheet_name = "SOE24-25"

# # Get the current date to use in the renamed sheet name
# date_str = datetime.now().strftime("%Y_%m_%d")
# renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# # Load the workbook and rename the sheet if it exists
# workbook = load_workbook(file_path)

# if original_sheet_name in workbook.sheetnames:
#     # Rename the existing sheet
#     sheet = workbook[original_sheet_name]
#     sheet.title = renamed_sheet_name
#     sheet.sheet_state = 'hidden'  # Hide the sheet

# # Save the workbook after renaming and hiding the sheet
# workbook.save(file_path)

# # Write the new data to the original sheet name
# with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
#     df_bank_match_soe.to_excel(writer, sheet_name=original_sheet_name, index=False)


In [190]:
# original_sheet_name = "ME24-25"

# # Get the current date to use in the renamed sheet name
# date_str = datetime.now().strftime("%Y_%m_%d")
# renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# # Load the workbook and rename the sheet if it exists
# workbook = load_workbook(file_path)

# if original_sheet_name in workbook.sheetnames:
#     # Rename the existing sheet
#     sheet = workbook[original_sheet_name]
#     sheet.title = renamed_sheet_name
#     sheet.sheet_state = 'hidden'  # Hide the sheet

# # Save the workbook after renaming and hiding the sheet
# workbook.save(file_path)

# # Write the new data to the original sheet name
# with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
#     df_bank_match_me.to_excel(writer, sheet_name=original_sheet_name, index=False)


In [191]:
# df_summary_all_bank_soe_due.to_excel("/content/drive/My Drive/2024_25/daybook.xlsx",sheet_name='SOE_overview')

# with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
#   df_soe.to_excel(writer,sheet_name='SOE_raw_xml')
# with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
#   df_summary_all_bank_me_due.to_excel(writer,sheet_name='ME_overview')
# with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
#   df_me.to_excel(writer,sheet_name='ME_raw_xml')